In [ ]:
import warnings, os
import datetime as dt
import itertools as it
import pandas as pd

warnings.filterwarnings('ignore')
pd.set_option('display.width',       None)
pd.set_option('display.max_columns', None)

In [ ]:
import MetaTrader5 as mt5

# Connection

In [ ]:
RICO_REAL   = { 'login':16331371,   'password':'#000000Zz', 'server':'Rico-PRD'                 }
CLEAR_REAL  = { 'login':1018156111, 'password':'#000000Zz', 'server':'ClearInvestimentos-CLEAR' }

PATH_1 = r'C:\Program Files\MetaTrader 5 _1\terminal64.exe'

assert mt5.initialize(PATH_1, **CLEAR_REAL)

# Variables

In [ ]:
SYMBOLS = ['WIN$N'] 
TFRAMES = ['M5','M15','H1','H3','D1','W1']

# Consume

In [ ]:
def MT5_LOAD_DATA(SYMBOLS, TFRAMES, start_pos=0, count=99_999):
    Pipe = []
    for (symbol, TF) in it.product(SYMBOLS, TFRAMES):
        timeframe = { 'M1':mt5.TIMEFRAME_M1, 'M5':mt5.TIMEFRAME_M5, 'M10':mt5.TIMEFRAME_M10, 'M15':mt5.TIMEFRAME_M15, 'M20':mt5.TIMEFRAME_M20, 'M30':mt5.TIMEFRAME_M30, 'H1':mt5.TIMEFRAME_H1, 'H2':mt5.TIMEFRAME_H2, 'H3':mt5.TIMEFRAME_H3, 'H4':mt5.TIMEFRAME_H4, 'H6':mt5.TIMEFRAME_H6, 'H8':mt5.TIMEFRAME_H8, 'H12':mt5.TIMEFRAME_H12, 'D1':mt5.TIMEFRAME_D1, 'W1':mt5.TIMEFRAME_W1 }[TF]
        Raw       = pd.DataFrame(mt5.copy_rates_from_pos(symbol, timeframe, start_pos, count))
        Raw.insert(0, 'a',          Raw.index + 1)
        Raw.insert(1, 'z',          Raw.index - Raw.index.max())
        Raw.insert(2, 'symbol',     symbol)
        Raw.insert(3, 'tf',         TF)
        Raw.insert(4, 'datetime',   pd.to_datetime(Raw['time'], unit='s'))
        Pipe.append(Raw)
    pass
    return pd.concat(Pipe, ignore_index=1)

Raw = MT5_LOAD_DATA(SYMBOLS, TFRAMES)
Raw.head()

In [ ]:
raise Exception('STOP')

# Save

In [ ]:
Raw.to_csv('Metatrader_WINN.csv', index=False)

# Load

In [ ]:
Raw = pd.read_csv('Raw.csv', parse_dates=['datetime'])
Raw.head()

In [12]:
def QUERY_SELECT(Raw, SYMBOLS, TFRAMES):
    return Raw[Raw['symbol'].isin(SYMBOLS) & Raw['tf'].isin(TFRAMES)] .reset_index(drop=1)

Query = QUERY_SELECT(Raw, SYMBOLS=[*SYMBOLS], TFRAMES=[*TFRAMES])
Query.head()

,a,z,symbol,tf,datetime,time,open,high,low,close,tick_volume,spread,real_volume
0,1,-99998,WIN$N,M5,2021-12-02 17:45:00,1638467100,104220.0,104250.0,104035.0,104190.0,40010,5,135191
1,2,-99997,WIN$N,M5,2021-12-02 17:50:00,1638467400,104190.0,104390.0,104115.0,104300.0,39363,5,137891
2,3,-99996,WIN$N,M5,2021-12-02 17:55:00,1638467700,104295.0,104520.0,104295.0,104495.0,36882,5,123994
3,4,-99995,WIN$N,M5,2021-12-02 18:00:00,1638468000,104490.0,104655.0,104445.0,104455.0,32889,5,115806
4,5,-99994,WIN$N,M5,2021-12-02 18:05:00,1638468300,104460.0,104600.0,104430.0,104525.0,21082,5,64871


# Check

In [13]:
Raw.groupby('tf', sort=0)[['datetime']].count()

,datetime
tf,
M5,99999
M15,46107
H1,11867
H3,4412
D1,1245
W1,261


In [14]:
Raw.groupby('tf', sort=0)[['datetime']].first()

,datetime
tf,
M5,2021-12-02 17:45:00
M15,2020-07-20 09:00:00
H1,2020-07-20 09:00:00
H3,2020-07-20 09:00:00
D1,2020-07-20 00:00:00
W1,2020-07-19 00:00:00
